# H2A Extraction, Preprocessing, and Cleaning

Install if needed (uncomment the following lines):

In [ ]:
#%pip install selenium webdriver-manager beautifulsoup4 pandas requests
#%pip install PyPDF2 #- > Read and parse your content pdf
#%pip install requests #- > request for get the pdf
#%pip install BeautifulSoup # - > for parse the html and find all url hrf with ".pdf" final
#%pip install pytesseract pdf2image
#%pip install geopy pandas

In [35]:
# Import necessary libraries
import pandas as pd
import numpy as np
import os
import time
import requests
import io
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import urljoin
from concurrent.futures import ThreadPoolExecutor
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urlparse
from PyPDF2 import PdfFileReader
from PyPDF2 import PdfReader
from pdf2image import convert_from_path
import pytesseract
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import GoogleV3

In [36]:
# Load the data from csv
data = pd.read_csv(
    "Master Document of H-2A Orders - Hit or Not - Previous Semester H-2A Orders Successes.csv"
)

# Print the first 5 rows of the data
print(data.head())

          Case Number                                       Link to H-2A  \
0  H-300-21280-631097  https://seasonaljobs.dol.gov/api/job-order/H-3...   
1  H-300-21202-474481  https://seasonaljobs.dol.gov/api/job-order/H-3...   
2  H-300-21195-461870  https://seasonaljobs.dol.gov/api/job-order/H-3...   
3  H-300-21183-441615  https://seasonaljobs.dol.gov/api/job-order/H-3...   
4  H-300-20255-817429  https://seasonaljobs.dol.gov/api/job-order/H-3...   

  Hit or No Hit Market Farm Narratives  \
0        No Hit    NaN  NaN        NaN   
1        No Hit    NaN  NaN        NaN   
2        No Hit    NaN  NaN        NaN   
3        No Hit    NaN  NaN        NaN   
4        No Hit    NaN  NaN        NaN   

  Facebook. Google Reviews, Website, LinkedIn, Google Maps, Instagram, etc.???  \
0                                                NaN                             
1                                                NaN                             
2                                           

In [37]:
# Make a df of just rows with "hit"
hit = data[data['Hit or No Hit'] == 'Hit']

# Print the first 5 rows of the hit data
print(hit.head())

           Case Number                                       Link to H-2A  \
13  H-300-21029-042637  https://seasonaljobs.dol.gov/api/job-order/H-3...   
35  H-300-20013-248635  https://seasonaljobs.dol.gov/api/job-order/H-3...   
38  H-300-20002-228933  https://seasonaljobs.dol.gov/api/job-order/H-3...   
44  H-300-19044-646927  https://seasonaljobs.dol.gov/jobs/H-300-19044-...   
48  H-300-19114-230380  https://seasonaljobs.dol.gov/jobs/H-300-19114-...   

   Hit or No Hit                                             Market  \
13           Hit                        Kroger, Walmart, Instacart    
35           Hit  Naturipe Farms, 99 Cents Only Stores, Ahold, A...   
38           Hit  Store Locator – sealtheseasons, Peoples-Food A...   
44           Hit                             https://wishfarms.com/   
48           Hit                                      Harris Teeter   

                             Farm  \
13                  Haigler Farms   
35         Down South Berries LLC   

In [38]:
# Save the hit data to a new csv
hit.to_csv('hit.csv', index=False)

In [39]:
file_path = "hit.csv"
df = pd.read_csv(file_path)

# Column containing URLs
column_name = "Link to H-2A"
urls = df[column_name].dropna().unique()

# Create a folder for PDFs
output_folder = "H2A_PDFs"
os.makedirs(output_folder, exist_ok=True)

# Set up Chrome WebDriver (Headless Mode)
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--log-level=3")

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# Track successes and failures
successful_downloads = []
failed_urls = []


# Function to check if a URL is a direct PDF
def is_pdf(url):
    try:
        response = requests.head(url, allow_redirects=True, timeout=10)
        content_type = response.headers.get("Content-Type", "")

        if not content_type or "application/pdf" not in content_type:
            response = requests.get(url, stream=True, timeout=10)
            content_type = response.headers.get("Content-Type", "")

        return "application/pdf" in content_type
    except requests.exceptions.RequestException:
        return False


# Function to extract PDF link from a webpage
def extract_pdf_from_page(url):
    try:
        driver.get(url)
        time.sleep(2)

        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )

        soup = BeautifulSoup(driver.page_source, "html.parser")

        # Try extracting direct PDF links
        for link in soup.find_all("a", href=True):
            text = link.text.strip().lower()
            if "view job order" in text or link["href"].endswith(".pdf"):
                return urljoin(url, link["href"])

        # Look for PDFs in <iframe>
        iframe = soup.find("iframe", src=True)
        if iframe and iframe["src"].endswith(".pdf"):
            return urljoin(url, iframe["src"])

    except Exception as e:
        print(f"Error processing webpage {url}: {e}")
    return None


# Function to download a PDF and track success
def download_pdf(pdf_url, source_url):
    try:
        response = requests.get(pdf_url, stream=True, timeout=10)
        content_type = response.headers.get("Content-Type", "")

        if "application/pdf" not in content_type:
            print(f"Not a PDF: {pdf_url} (Content-Type: {content_type})")
            failed_urls.append(source_url)
            return

        filename = os.path.basename(pdf_url)
        if not filename.endswith(".pdf"):
            filename += ".pdf"

        file_path = os.path.join(output_folder, filename)

        # Write PDF file
        with open(file_path, "wb") as file:
            for chunk in response.iter_content(chunk_size=1024):
                file.write(chunk)

        if os.path.exists(file_path) and os.path.getsize(file_path) > 0:
            print(f"Downloaded: {file_path}")
            successful_downloads.append(file_path)
        else:
            print(f"Failed to save PDF properly: {file_path}")
            failed_urls.append(source_url)

    except requests.exceptions.RequestException as e:
        print(f"Error downloading PDF from {source_url}: {e}")
        failed_urls.append(source_url)


# Function to process each URL
def process_url(url):
    global driver
    try:
        if is_pdf(url):
            download_pdf(url, url)
        else:
            pdf_url = extract_pdf_from_page(url)
            if pdf_url:
                download_pdf(pdf_url, url)
            else:
                print(f"No 'View Job Order' PDF found on webpage: {url}")
                failed_urls.append(url)

    except Exception as e:
        print(f"WebDriver error, restarting: {e}")
        driver.quit()
        driver = webdriver.Chrome(service=service, options=chrome_options)


# Use multi threading for faster downloads
with ThreadPoolExecutor(max_workers=3) as executor:
    executor.map(process_url, urls)

# Close the Selenium browser
driver.quit()

# Print final download summary
print("\n**Download Summary**")
print(f"Successfully downloaded PDFs: {len(successful_downloads)}")
print(f"Failed URLs: {len(failed_urls)}\n")

if successful_downloads:
    print("**Successfully downloaded PDFs:**")
    for file in successful_downloads:
        print(file)

if failed_urls:
    print("\n**Failed to download PDFs from these URLs:**")
    for url in failed_urls:
        print(url)

# Debugging output
print("\nDebugging successful_downloads list:")
print(successful_downloads)

Downloaded: H2A_PDFs/H-300-21029-042637.pdf
Downloaded: H2A_PDFs/H-300-20013-248635.pdf
Not a PDF: https://seasonaljobs.dol.gov/api/job-order/H-300-20002-228933 (Content-Type: application/json; charset=utf-8)
Not a PDF: https://seasonaljobs.dol.gov/jobs/H-300-19044-646927 (Content-Type: text/html; charset=utf-8)
Not a PDF: https://seasonaljobs.dol.gov/jobs/H-300-19114-230380 (Content-Type: text/html; charset=utf-8)
Downloaded: H2A_PDFs/H-300-22014-831909.pdf
Downloaded: H2A_PDFs/H-300-20357-975271.pdf
Downloaded: H2A_PDFs/H-300-21006-998351.pdf
Downloaded: H2A_PDFs/H-300-20009-241352.pdf
Not a PDF: https://seasonaljobs.dol.gov/jobs/H-300-19068-891355 (Content-Type: text/html; charset=utf-8)
Not a PDF: https://seasonaljobs.dol.gov/jobs/H-300-19106-808682 (Content-Type: text/html; charset=utf-8)
Not a PDF: https://seasonaljobs.dol.gov/jobs/H-300-19051-356435 (Content-Type: text/html; charset=utf-8)
Not a PDF: https://seasonaljobs.dol.gov/jobs/H-300-18254-022527 (Content-Type: text/html; 

In [40]:
file_path = "hit.csv"
df = pd.read_csv(file_path)

# Extract URLs from the "Link to H-2A" column
column_name = "Link to H-2A"
urls = df[column_name].dropna().unique()

# Extract expected filenames from URLs
expected_filenames = []
for url in urls:
    parsed_url = urlparse(url)
    filename = os.path.basename(parsed_url.path)
    if not filename.endswith(".pdf"):
        filename += ".pdf"  # Ensure consistency with saved files
    expected_filenames.append(filename)

# Get downloaded files in H2A_PDFs/
downloaded_files = set(os.listdir("H2A_PDFs"))

# Identify missing files
missing_files = [filename for filename in expected_filenames if filename not in downloaded_files]

# Print results
print("\n**Download Comparison**")
print(f"Total URLs in hit.csv: {len(expected_filenames)}")
print(f"Total PDFs downloaded: {len(downloaded_files)}")
print(f"Missing PDFs: {len(missing_files)}\n")

if missing_files:
    print("**Missing PDFs (Expected but Not Found):**")
    for filename in missing_files:
        print(filename)
else:
    print("All expected PDFs were downloaded!")


**Download Comparison**
Total URLs in hit.csv: 33
Total PDFs downloaded: 140
Missing PDFs: 7

**Missing PDFs (Expected but Not Found):**
H-300-19044-646927.pdf
H-300-19114-230380.pdf
H-300-19068-891355.pdf
H-300-19106-808682.pdf
H-300-19051-356435.pdf
H-300-18254-022527.pdf
H-300-23028-734237.pdf


We know 2 links are broken:
  1. H-300-19106-808682
  2. H-300-23028-734237


That leaves 5 missing PDFs to account for.

#### Attempt to use BeautifulSoup to scrape information from the PDFs

In [42]:
# Define the folder path and output CSV file
pdf_folder = "H2A_PDFs"
output_csv = "extracted_text.csv"

# List all PDF files in the folder
pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith('.pdf')]

# Store extracted data
data = []

# Loop through and process each PDF
for pdf_file in pdf_files:
    pdf_path = os.path.join(pdf_folder, pdf_file)

    try:
        with open(pdf_path, "rb") as file:
            reader = PdfReader(file)
            
            # Extract text from each page
            text = "\n".join([page.extract_text() for page in reader.pages if page.extract_text()])
            
            # Append data to list
            data.append({"filename": pdf_file, "text": text})

    except Exception as e:
        print(f"Error processing {pdf_file}: {e}")

# Convert list to df
df = pd.DataFrame(data)

# Save to CSV
df.to_csv(output_csv, index=False, encoding='utf-8')

print(f"Extracted text saved to {output_csv}")


Error processing 117168_order.pdf: [Errno 22] Invalid argument
Error processing 116897_order.pdf: [Errno 22] Invalid argument
Error processing 110349_order.pdf: [Errno 22] Invalid argument


unknown widths : 
[0, IndirectObject(216, 0, 5668848784)]
unknown widths : 
[0, IndirectObject(219, 0, 5668848784)]
unknown widths : 
[0, IndirectObject(222, 0, 5668848784)]
unknown widths : 
[0, IndirectObject(225, 0, 5668848784)]
unknown widths : 
[0, IndirectObject(228, 0, 5668848784)]
unknown widths : 
[0, IndirectObject(231, 0, 5668848784)]
unknown widths : 
[0, IndirectObject(234, 0, 5668848784)]
unknown widths : 
[0, IndirectObject(237, 0, 5668848784)]
unknown widths : 
[0, IndirectObject(240, 0, 5668848784)]
unknown widths : 
[0, IndirectObject(243, 0, 5668848784)]
unknown widths : 
[0, IndirectObject(246, 0, 5668848784)]
unknown widths : 
[0, IndirectObject(249, 0, 5668848784)]
unknown widths : 
[0, IndirectObject(252, 0, 5668848784)]
unknown widths : 
[0, IndirectObject(255, 0, 5668848784)]
unknown widths : 
[0, IndirectObject(258, 0, 5668848784)]
unknown widths : 
[0, IndirectObject(261, 0, 5668848784)]
unknown widths : 
[0, IndirectObject(264, 0, 5668848784)]
unknown widths

Extracted text saved to extracted_text.csv


In [43]:
# Define the folder path where HTML files are stored
html_folder = "H2A_PDFs"  

# List all HTML files in the folder
html_files = [f for f in os.listdir(html_folder) if f.endswith('.html')]

# Loop through and process each HTML file
for html_file in html_files:
    html_path = os.path.join(html_folder, html_file)

    # Open and read the HTML file
    with open(html_path, "r", encoding="utf-8") as file:
        soup = BeautifulSoup(file, "html.parser")

    print(f"\nExtracting data from: {html_file}\n")
   # Extract text content
    text = " ".join([p.text for p in soup.find_all('p')])
    links = [a['href'] for a in soup.find_all('a', href=True)]

    data.append({"filename": html_file, "text": text, "links": "; ".join(links)})

# Convert to df and save
df = pd.DataFrame(data)
df.to_csv("extracted_html_data.csv", index=False, encoding="utf-8")

print("Extracted data saved to extracted_html_data.csv")


Extracted data saved to extracted_html_data.csv


#### Use Tesseract for scanned images to help get information

In [45]:
# Convert PDF to images
images = convert_from_path(
    "H2A_PDFs/66774_order.pdf", 
    dpi=300, poppler_path="/opt/homebrew/bin/", fmt="pdf")

In [46]:
pytesseract.pytesseract.tesseract_cmd = "/opt/homebrew/bin/tesseract"

for i, img in enumerate(images):
    text = pytesseract.image_to_string(img)
    print(f"--- Page {i+1} ---\n{text}\n")

--- Page 1 ---
U.S. Department Lapor

(Print or type in each field block - To include additional information, go to block # 28 - Please follow Step-By-Step instructions)
(Favor de usar letra de molde en la solicitud ~ Para incluir Informacion adicional vea el punto # 28 ~ Favor de seguir las instrucciones paso-a-pasp)

and Zip Code / Nombre y Direccidn del Empleador/Patron y/o Agente
(Numero, Calle, Ciudad, Estado y Codigo Postal ):

EMILIA ALVAREZ

233N LANIER ST

LYONS, GA 30436

a) Federal Employer identification Number (FEIN) / N&mero federa! de
Identificacién del Empleador:

b) Telephone Number / Nimero de Teléfono:
(843) 908-7328

¢) Fax Number / Numero de Fax:
NIA

d) E-mail Address / Direocién de Correo Electrinico: é
NIA

Sn Cy a al
3. Address and Directions to Work Site / Domicilio y Direcciones al lugar de

trabajo:
B&W BURCH FARMS
1556 STANFIELD ROAD
SCREVEN, GA 31560

PLEASE SEE ATTACHMETN FOR DIRECTION

3, Address and Directions to Housing / Domicilio y Direcciones al lug

In [47]:
text = pytesseract.image_to_string(img, lang='eng')

In [48]:
# Point to tesseract binary if needed
pytesseract.pytesseract.tesseract_cmd = "/opt/homebrew/bin/tesseract"

# Convert PDF to images
pdf_path = "H2A_PDFs/66774_order.pdf"
images = convert_from_path(pdf_path, dpi=300, poppler_path="/opt/homebrew/bin")

# Define a basic text cleaner
def clean_text(text):
    text = text.replace("\n", " ")                      # flatten newlines
    text = re.sub(r'\s{2,}', ' ', text)                 # remove extra spaces
    text = re.sub(r'[^ -~\n]+', '', text)               # remove weird characters
    text = text.strip()
    return text

# OCR + Clean each page
all_text = ""
for i, img in enumerate(images):
    raw_text = pytesseract.image_to_string(img)
    cleaned = clean_text(raw_text)
    all_text += f"\n--- Page {i + 1} ---\n{cleaned}\n"

# Save to file
output_path = "66774_order_cleaned.txt"
with open(output_path, "w") as f:
    f.write(all_text)

print(f"Done! Cleaned text saved to: {output_path}")


Done! Cleaned text saved to: 66774_order_cleaned.txt


In [49]:
# Point to tesseract binary if needed
pytesseract.pytesseract.tesseract_cmd = "/opt/homebrew/bin/tesseract"

# List of PDF filenames
pdf_filenames = [
    "74479_order.pdf", "99257_order.pdf", "104779_order.pdf",
    "110349_order.pdf", "115829_order.pdf", "115969_order.pdf",
    "116897_order.pdf", "117168_order.pdf", "117179_order.pdf"
]

# Folder where PDFs are stored
pdf_folder = "H2A_PDFs"

# Output folder
output_folder = "ocr_outputs"
os.makedirs(output_folder, exist_ok=True)

# Text cleaning function
def clean_text(text):
    text = text.replace("\n", " ")
    text = re.sub(r'\s{2,}', ' ', text)
    text = re.sub(r'[^ -~\n]+', '', text)
    return text.strip()

# Loop through each PDF
for filename in pdf_filenames:
    pdf_path = os.path.join(pdf_folder, filename)
    print(f"Processing: {filename}")

    try:
        images = convert_from_path(pdf_path, dpi=300, poppler_path="/opt/homebrew/bin")
        all_text = ""
        
        for i, img in enumerate(images):
            raw_text = pytesseract.image_to_string(img)
            cleaned = clean_text(raw_text)
            all_text += f"\n--- Page {i + 1} ---\n{cleaned}\n"

        # Save to a text file
        output_file = os.path.join(output_folder, filename.replace(".pdf", "_cleaned.txt"))
        with open(output_file, "w") as f:
            f.write(all_text)
        print(f"Saved to {output_file}")

    except Exception as e:
        print(f"Failed to process {filename}: {e}")


Processing: 74479_order.pdf
Saved to ocr_outputs/74479_order_cleaned.txt
Processing: 99257_order.pdf
Saved to ocr_outputs/99257_order_cleaned.txt
Processing: 104779_order.pdf
Saved to ocr_outputs/104779_order_cleaned.txt
Processing: 110349_order.pdf
Failed to process 110349_order.pdf: Unable to get page count.
Syntax Error: Couldn't find trailer dictionary
Syntax Error: Invalid XRef entry 385
Syntax Error: Invalid XRef entry 380
Syntax Error: Top-level pages object is wrong type (null)
Command Line Error: Wrong page range given: the first page (1) can not be after the last page (0).

Processing: 115829_order.pdf
Saved to ocr_outputs/115829_order_cleaned.txt
Processing: 115969_order.pdf
Saved to ocr_outputs/115969_order_cleaned.txt
Processing: 116897_order.pdf
Failed to process 116897_order.pdf: Unable to get page count.
Syntax Error: Couldn't find trailer dictionary
Syntax Error: Invalid XRef entry 344
Syntax Error: Invalid XRef entry 343
Syntax Error: Top-level pages object is wrong t

#### Sample pattern to pull farm information


In [50]:
farm_names = set()
text_folder = "ocr_outputs"

# Pattern to catch likely farm names
patterns = [
    r"(?i)(?:employer|grower|farm name)[:\s]+([A-Z][A-Za-z0-9,&.\'’\-\s]+)",  # after keywords
    r"\b([A-Z][A-Za-z0-9,&.\'’\-]+\s+(?:Farms?|Orchards?|Ranch|Nursery|Growers?|Ag(?:riculture)?|LLC|Inc\.?))\b"  # freestanding names
]

for file in os.listdir(text_folder):
    if file.endswith(".txt"):
        with open(os.path.join(text_folder, file)) as f:
            text = f.read()
            for pattern in patterns:
                matches = re.findall(pattern, text)
                farm_names.update([match.strip() for match in matches])

# View the unique farm names
for name in sorted(farm_names):
    print(name)


AG, LLC
Ag Labor LLC
Assurance Ag
Assurance Ag Labor LLC with crops in the counties of Hillsborough in Florida hereby agrees to comply by the regulations of 20 CRF 655,135 and 20 CRF 858.122
Assurance Ag Labor LLC with crops in the counties of Hillsborough in Florida hereby agrees to comply by the regulations of 20 CRF 655.135 and 20 CRF 655.122
Assurance J and J Harvesting, LLC with crops in the counties of Terrell, Turner and Worth in Georgia hereby agrees to comply by the regulations of 20 CRF 655.135 and 20 CRE 655.122
Blueberry Farm
COCORDIA, LLC
Co, Inc
Crop, Nursery
DATE

--- Page 19 ---
CONTRATO PARA EL TRABAJO AGRICULTURAL Empleador Empleado El empleado asignado se compromete fielmente a todos los trminos y las condiciones para llevar a cabo todas las tareas asignadas dentro de la descripcion del trabajo bajo la direccion del empresario segun lo establecido. Este empleo se regira por las siguientes
EMILIA ALVAREZ 233 LANIER ST LYONS, GA 30436
EMILIA ALVAREZ 233 N LANIER ST LYO

#### Conversion of CSV to text file to extract farm information from files

In [51]:
df = pd.read_csv("extracted_html_data.csv")

# Save as .txt (comma separated)
df.to_csv("extracted_html_data.txt", index=False, sep=',')

### Regular expressions to identify addresses and save them in a file

- This is for the scanned image files in the ocr_outputs folder

In [52]:
folder_path = "ocr_outputs"

# Regex pattern to extract addresses
address_pattern = r'\d{1,6}\s+[A-Za-z0-9.,&\-\' ]+(?:St|Street|Ave|Avenue|Blvd|Boulevard|Rd|Road|Ln|Lane|Dr|Drive|Ct|Court|Pl|Place|Hwy|Highway|Way)\s+[A-Za-z ]+,\s+[A-Z]{2}\s+\d{5}'

# Collect all addresses
all_addresses = []

# Loop through all .txt files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, "r") as f:
            text = f.read()
            matches = re.findall(address_pattern, text)
            for match in matches:
                all_addresses.append((filename, match.strip()))

# Print results
for file, address in all_addresses:
    print(f"{file}: {address}")

# Save to .txt
with open("extracted_addresses_ocr.txt", "w") as out:
    for file, address in all_addresses:
        out.write(f"{file}: {address}\n")

99257_order_cleaned.txt: 202 Knights Griffin Rd Plant City, FL 33565 501 Half Mile Rd Plant City, FL 33565 3306 N Wilder Rd Plant City, FL 33563 3910 N Wilder Rd Plant City, FL 33565 5004 E Knights Griffin Rd, Plant City, FL 33565 4404 W Knights Griffin Rd Plant City, FL 33565 9211 Kathleen Rd Lakeland, FL 33810
99257_order_cleaned.txt: 6 Merrion Rd Lakeland, FL 33810 3838 Fancy Farms Rd Plant City, Fl 33566 5212 Drane Field Rd Plant City, Fl 33566 4109 Futeh Loop Plant City, FL 33566 5112 Mud Lake Rd Plant City, FL 33567 416 Jerry Smith Rd Dover, FL 33527
99257_order_cleaned.txt: 2702 W Knights Griffin Rd Plant City, FL 33562 2706 W Knights Griffin Re Plant City, FL 33562 Ranch Camp Loop Plant City, FL 33565 102 Kaysville Rd Lithia, FL 33547
99257_order_cleaned.txt: 107 East Madison Street Tallahassee, FL 32399
99257_order_cleaned.txt: 5102 Miley Rd Plant City, FL 33565
99257_order_cleaned.txt: 4407 Spotted Horse Lane Plant City, FL 33565
99257_order_cleaned.txt: 5021 W Knights Griffi

#### Now this function will run through for the extracted_html_data.txt file

In [53]:
# Load the text
with open("extracted_html_data.txt", "r") as f:
    text = f.read()

# Address like pattern
address_pattern = r'\d{1,6}\s?[A-Z0-9][\w\s.&\-]+(?:ST|STREET|AVE|AVENUE|BLVD|HWY|ROAD|RD|LANE|LN|DR|DRIVE|COURT|CT|PL|PLACE|HWY|HIGHWAY)?[,\s]+[A-Z]+\s?[A-Z]{2,3}[,\s]+\d{5}'

# Pull matches
matches = re.findall(address_pattern, text, re.IGNORECASE)

# Clean and deduplicate
addresses = sorted(set([match.strip() for match in matches]))

# Output
for address in addresses:
    print(address)

# Save to .txt
with open("extracted_addresses.txt", "w") as out_file:
    for address in addresses:
        out_file.write(address + "\n")

1002 Smith Ave
Douglas
Georgia
31533
1002 Smith Ave.
Douglas
Georgia
31533
102 Joe Dasher Rd
Glennville, Georgia 30427
10356 Cogdell Hwy
Pearson, Georgia 31642
106 Seashell Road
Alma
Georgia
31510
1104 Maynard Ave.
Dalhart
Texas
79022
113 Dorothy Burnam Rd
Nicholls
Georgia
31554
120 GA Hwy 196
Glennville, Georgia 30427
123 Liberal Street
Dalhart, Texas 79022
1250 Interocean Drive
Holyoke
Colorado
80734
147 PINEVIEW ROAD
180 OCELOT ROAD
Alma
Georgia
31510
18 ALAN DRIVE
DOUGLAS
Georgia
31535
18 Alan Drive
Douglas
Georgia
31535
183 MARIGOLD LANE
ALMA
Georgia
31510
1935 Ross Bowen Road
Lyons
Georgia
30436
20
Glennville
Georgia
30427
201 4th Avenue NW
Ruskin
Florida
33570
2025
Once Hired worker may be required to submit a random drug test at no cost to worker. Testing
positive or failure to comply may result in immediate termination from employment.
G & R Farms 102  Dasher Rd
Glennville
Georgia
30427
205-3441
111
Barton
Farmer
321 Twin Oaks
Nicholls
Georgia
31554
213 PINECONE ROAD
Alma
Geor

#### For extracted_addresses.txt (which has a different format than the previous two)

In [54]:
# Load the raw text file
with open("extracted_addresses.txt", "r") as f:
    lines = [line.strip() for line in f if line.strip()]

grouped_addresses = []
current_block = []

# Keywords to help us split/group logically
state_zip_pattern = re.compile(r"\b[A-Z]{2}\s*\d{5}$", re.IGNORECASE)
city_state_zip_pattern = re.compile(r".+,\s*[A-Za-z]{2,}\s+\d{5}$")

for line in lines:
    current_block.append(line)

    # If line contains city, state ZIP or just ends in ZIP, it's likely the end of an address block
    if state_zip_pattern.search(line) or city_state_zip_pattern.search(line):
        grouped_addresses.append(" ".join(current_block))
        current_block = []

# Handle any leftover lines
if current_block:
    grouped_addresses.append(" ".join(current_block))

# Save to a cleaned output file
with open("individual_cleaned_addresses.txt", "w") as out:
    for addr in grouped_addresses:
        out.write(addr + "\n")

print(f"{len(grouped_addresses)} addresses written to 'individual_cleaned_addresses.txt'")

16 addresses written to 'individual_cleaned_addresses.txt'


In [55]:
# Load the file
with open("individual_cleaned_addresses.txt", "r") as f:
    text = f.read()

# Regex pattern to extract address like chunks
# This looks for 3+ words ending in a ZIP code (5 digits)
address_pattern = re.compile(
    r'(?:(?:[A-Z][a-z]*\s)*\d{1,6}[\w\s\.\-]*?(?:Road|Rd|Ave|Avenue|St|Street|Lane|Ln|Drive|Dr|Blvd|Boulevard|Way)?\s+)?[A-Z][a-z]+(?:,\s*)?\s+[A-Z][a-z]+(?:,\s*)?\s+\d{5}',
    re.IGNORECASE
)

# Extract all matches
matches = re.findall(address_pattern, text)

# Deduplicate and clean
unique_addresses = sorted(set(m.strip() for m in matches))

# Save results
with open("final_cleaned_addresses.txt", "w") as f:
    for addr in unique_addresses:
        f.write(addr + "\n")

print(f"Extracted {len(unique_addresses)} individual addresses.")


Extracted 49 individual addresses.


In [56]:
# Load and combine both text files
with open("extracted_addresses_html.txt", "r") as f1, open("extracted_addresses_ocr.txt", "r") as f2:
    combined_text = f1.read() + "\n" + f2.read()

# Define address pattern (US addresses ending in city, state ZIP)
address_pattern = r'\d{1,6}[\w\s\.,\-]*?\s+[A-Za-z\s]+,\s*[A-Z]{2}\s*\d{5}'

# Extract all matches
addresses = re.findall(address_pattern, combined_text)

# Clean and deduplicate
cleaned_addresses = sorted(set(addr.strip() for addr in addresses))

# Save to a new txt file
with open("merged_cleaned_addresses.txt", "w") as f_out:
    for addr in cleaned_addresses:
        f_out.write(addr + "\n")

print(f"Done! {len(cleaned_addresses)} addresses saved to 'merged_cleaned_addresses.txt'")

Done! 70 addresses saved to 'merged_cleaned_addresses.txt'


#### Geocoding Information

In [57]:
# Load cleaned address file
with open("final_cleaned_addresses.txt", "r") as f:
    raw_lines = [line.strip() for line in f if line.strip()]

# Keep only lines that end with a ZIP code
zip_pattern = re.compile(r'\b\d{5}\b')
address_lines = [line for line in raw_lines if zip_pattern.search(line)]

# Remove phone numbers and extra roles
def clean_address(line):
    line = re.sub(r'\b\d{3}[-.\s]?\d{4}\b', '', line)  # remove phone numbers
    line = re.sub(r'\b(Farm Labor Contractor|Owner|Contractor|Farmer|R Farms)\b', '', line, flags=re.IGNORECASE)
    return line.strip()

cleaned_addresses = [clean_address(line) for line in address_lines]

# Set up geocoder
geolocator = Nominatim(user_agent="address_mapper")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# Geocode each address
results = []
for original, cleaned in zip(address_lines, cleaned_addresses):
    try:
        location = geocode(cleaned)
        if location:
            results.append({
                "original_address": original,
                "cleaned_address": cleaned,
                "latitude": location.latitude,
                "longitude": location.longitude
            })
    except Exception:
        results.append({
            "original_address": original,
            "cleaned_address": cleaned,
            "latitude": None,
            "longitude": None
        })

# Save to CSV
df = pd.DataFrame(results)
df.to_csv("geocoded_addresses.csv", index=False)
print("Done! Saved to geocoded_addresses.csv")

RateLimiter caught an error, retrying (0/2 tries). Called with (*('106 Seashell Road Alma Georgia 31510',), **{}).
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py", line 536, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connection.py", line 461, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/http/client.py", line 1378, in getresponse
    response.begin()
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/http/client.py", line 318, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/p

Done! Saved to geocoded_addresses.csv


#### Extract the case number and address from each job order in extracted_text.csv

In [58]:
# Load extracted text
df = pd.read_csv("extracted_text.csv")
df = df[~df["filename"].str.endswith("_order.pdf")].copy()
df["case_number"] = df["filename"].str.extract(r"(H-\d{3}-\d{5}-\d{6})")
df["file_id"] = df["filename"].str.replace(".pdf", "", regex=False)


# Load fallback address files
def load_fallback_file(path):
    d = {}
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            if ":" in line:
                name, addr = line.split(":", 1)
                d[name.strip().lower()] = addr.strip()
    return d


fallback_final = load_fallback_file("final_cleaned_addresses.txt")
fallback_html = load_fallback_file("extracted_addresses_html.txt")
fallback_ocr = load_fallback_file("extracted_addresses_ocr.txt")

# Address logic
address_keywords = [
    "road",
    "rd",
    "street",
    "st",
    "lane",
    "ln",
    "drive",
    "dr",
    "blvd",
    "hwy",
    "highway",
    "ave",
    "trail",
    "way",
    "route",
    "fm",
]
states = [
    "AL",
    "AK",
    "AZ",
    "AR",
    "CA",
    "CO",
    "CT",
    "DE",
    "FL",
    "GA",
    "HI",
    "ID",
    "IL",
    "IN",
    "IA",
    "KS",
    "KY",
    "LA",
    "ME",
    "MD",
    "MA",
    "MI",
    "MN",
    "MS",
    "MO",
    "MT",
    "NE",
    "NV",
    "NH",
    "NJ",
    "NM",
    "NY",
    "NC",
    "ND",
    "OH",
    "OK",
    "OR",
    "PA",
    "RI",
    "SC",
    "SD",
    "TN",
    "TX",
    "UT",
    "VT",
    "VA",
    "WA",
    "WV",
    "WI",
    "WY",
]
block_terms = [
    "united states",
    "form",
    "eta",
    "labor",
    "general instructions",
    "page",
    "http",
    "electronically",
]
bad_value_terms = [
    "apartment",
    "suite",
    "floor",
    "province",
    "country",
    "section",
    "part",
]


# Structural rules
def is_strict_address(line):
    if any(term in line.lower() for term in block_terms):
        return False
    has_number = bool(re.match(r"^\d{1,6} ", line))
    has_keyword = any(k in line.lower() for k in address_keywords)
    has_state = any(state in line for state in states)
    has_zip = bool(re.search(r"\d{5}\b", line))
    return has_number and has_keyword and (has_state or has_zip)


def is_partial_address(line):
    if any(term in line.lower() for term in block_terms):
        return False
    return bool(re.match(r"^\d{1,6} ", line)) and any(
        k in line.lower() for k in address_keywords
    )


def is_city_state_zip(line):
    return any(state in line for state in states) or bool(re.search(r"\d{5}\b", line))


def looks_like_valid_value(line):
    if any(bad in line.lower() for bad in bad_value_terms):
        return False
    return bool(re.match(r"^\d{1,6} ", line)) and any(
        k in line.lower() for k in address_keywords
    )


# Field-based reconstruction


def reconstruct_from_fields(lines):
    for i in range(len(lines) - 1):
        l1 = lines[i].lower()
        l2 = lines[i + 1].strip()
        if "address 1" in l1 and looks_like_valid_value(l2):
            for j in range(i + 2, len(lines)):
                l3 = lines[j].strip()
                if is_city_state_zip(l3):
                    return l2 + ", " + l3 + " [form field reconstruction]"
    return ""


# Address extractor with fallback
near_misses = []


def extract_best_address(text, filename):
    if pd.isna(text):
        return ""
    lines = [line.strip() for line in text.split("\n") if line.strip()]
    for i in range(len(lines)):
        line = lines[i]
        if is_strict_address(line):
            return line + " [structured]"
        elif is_partial_address(line):
            next_line = lines[i + 1] if i + 1 < len(lines) else ""
            if is_city_state_zip(next_line):
                return line + " " + next_line + " [relaxed combo]"
            else:
                near_misses.append({"filename": filename, "line": line})
        elif any(k in line.lower() for k in address_keywords):
            near_misses.append({"filename": filename, "line": line})
    reconstructed = reconstruct_from_fields(lines)
    if reconstructed:
        return reconstructed
    if near_misses:
        for m in near_misses:
            if looks_like_valid_value(m["line"]):
                return m["line"] + " [near miss salvage]"
    return ""


# Combine with fallback


def get_best_address(row):
    from_text = extract_best_address(row["text"], row["filename"])
    if from_text:
        return from_text
    fid = row["file_id"].lower()
    for fb in [fallback_final, fallback_html, fallback_ocr]:
        addr = fb.get(fid)
        if addr and is_strict_address(addr):
            return addr + " [fallback structured]"
    return ""


# Run and save
df["best_address"] = df.apply(get_best_address, axis=1)
df[["filename", "case_number", "best_address"]].to_csv(
    "case_number_and_best_address_final.csv", index=False
)

# Export near misses
near_df = pd.DataFrame(near_misses)
if not near_df.empty:
    near_df.to_csv("address_near_misses.csv", index=False)

df[["filename", "case_number", "best_address"]].head()

,filename,case_number,best_address
0,H-300-20150-611826.pdf,H-300-20150-611826,1104 Maynard Ave. [near miss salvage]
2,H-300-22014-831909.pdf,H-300-22014-831909,"211 Dorothy Burnham Road Nicholls, Georgia 315..."
3,H-300-21039-063180.pdf,H-300-21039-063180,
4,H-300-21253-577257.pdf,H-300-21253-577257,36635 A Street [structured]
5,H-300-21124-285763.pdf,H-300-21124-285763,1104 Maynard Ave. [near miss salvage]


#### Cleaning and Standardizing

In [59]:
df = pd.read_csv("case_number_and_best_address_final.csv")

# Drop rows with missing or blank best_address
df_cleaned = df[df["best_address"].notna() & (df["best_address"].str.strip() != "")]

# Save to new file
df_cleaned.to_csv("case_number_and_best_address_cleaned.csv", index=False)

In [60]:
df = pd.read_csv("case_number_and_best_address_cleaned.csv")

# Remove tags like [structured], [relaxed combo], etc.
df["best_address"] = df["best_address"].str.replace(r"\\s*\\[.*\\]$", "", regex=True)

# Dictionary of known patterns to append missing city/state/ZIP (from previous extraction)
fixes = {
    "1104 Maynard Ave.": "Nicholls, GA 31554",
    "36635 A Street": "Wray, CO 80758",
    "213 PINECONE ROAD": "Alma, GA 31510",
    "931 SA HARDEN RD": "Lyons, GA 30436",
    "183 MARIGOLD LANE": "Alma, GA 31510"
}

# Apply corrections
df["best_address"] = df["best_address"].apply(
    lambda x: x if any(c in x for c in [",", " GA", " TX", " CO", "FL", "NY"]) else
    next((f"{x}, {fixes[k]}" for k in fixes if k.lower() in x.lower()), x)
)

# Save updated file
df.to_csv("case_number_and_best_address_geocodable.csv", index=False)

In [61]:
df["best_address"] = df["best_address"].str.replace(r"\s*\[.*?\]", "", regex=True).str.strip()
df.to_csv("case_number_and_best_address_geocodable.csv", index=False)

In [62]:
df["cleaned_address"] = df["best_address"].str.replace(r"\.", "", regex=True).str.replace(",", "").str.strip()

#### Geocoding

In [63]:
df = pd.read_csv("case_number_and_best_address_geocodable.csv")

# Initialize Google Maps geocoder
geolocator = GoogleV3(API_KEY = st.secrets["api_key"]) # Hide in github upload
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# Geocode the addresses
df["location"] = df["best_address"].apply(geocode)
df["latitude"] = df["location"].apply(lambda loc: loc.latitude if loc else None)
df["longitude"] = df["location"].apply(lambda loc: loc.longitude if loc else None)

# Save results
df.to_csv("h2a_google_geocoded_results.csv", index=False)

#### Merging New Dataset with Master

In [64]:
# Load both files
master_df = pd.read_csv("Master Document of H-2A Orders - Hit or Not - Previous Semester H-2A Orders Successes.csv")
geo_df = pd.read_csv("h2a_google_geocoded_results.csv")

# Normalize merge keys
master_df["Case Number"] = master_df["Case Number"].astype(str).str.strip()
geo_df["case_number"] = geo_df["case_number"].astype(str).str.strip()

# Merge by case number
df = master_df.merge(geo_df, left_on="Case Number", right_on="case_number", how="left")

# Drop rows with missing best_address
df = df[df["best_address"].notna() & (df["best_address"].str.strip() != "")]

# Drop unnecessary columns
df = df.drop(columns=["Narratives", "filename", "case_number", "Hits - By name or by logo"], errors="ignore")

# Clean column names for consistency
df.columns = (
    df.columns.str.strip()
              .str.lower()
              .str.replace(" ", "_")
              .str.replace(r"[^\w_]", "", regex=True)
)

# Rename confusing columns
df = df.rename(columns={
    "facebook_google_reviews_website_linkedin_google_maps_instagram_etc": "online_presence",
    "hits_by_name_or_by_logo": "hit_type"
})

# Standardize numeric columns
df["year"] = pd.to_numeric(df["year"], errors="coerce")
df["certainty"] = pd.to_numeric(df["certainty"], errors="coerce")

# Reorder key columns
preferred_order = [
    "case_number", "link_to_h2a", "farm", "market", "best_address", 
    "latitude", "longitude", "recruiter", "year", "hit_type", "person"
]
df = df[[col for col in preferred_order if col in df.columns] + [col for col in df.columns if col not in preferred_order]]

# Save cleaned file
df.to_csv("h2a_master_linked_with_geocodes.csv", index=False)

# Preview
print(df[["case_number", "best_address", "latitude", "longitude"]].head())

           case_number                                      best_address  \
35  H-300-20013-248635             1104 Maynard Ave., Nicholls, GA 31554   
38  H-300-20002-228933                 213 PINECONE ROAD, Alma, GA 31510   
68  H-300-22014-831909  211 Dorothy Burnham Road Nicholls, Georgia 31554   
73  H-300-20357-975271                 931 SA HARDEN RD, Lyons, GA 30436   
75  H-300-21006-998351                 183 MARIGOLD LANE, Alma, GA 31510   

     latitude  longitude  
35  31.517422 -82.634860  
38  31.499044 -82.500469  
68  31.490785 -82.631934  
73  32.135268 -82.313271  
75  31.485531 -82.444244  


In [65]:
# Load the merged geocoded dataset
df = pd.read_csv("h2a_master_linked_with_geocodes.csv")

# Clean column names
df.columns = (
    df.columns.str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace(r"[^\w_]", "", regex=True)
)

# Standardize data types
df["certainty"] = pd.to_numeric(df["certainty"], errors="coerce")
df["year"] = pd.to_numeric(df["year"], errors="coerce")

# Fill missing values
df["certainty"].fillna(0, inplace=True)
df["online_presence"].fillna("Unknown", inplace=True)
df["market"] = df["market"].str.replace("\n", "; ", regex=False)

# Add helper columns
df["has_facebook"] = df["online_presence"].str.contains(
    "facebook", case=False, na=False
)
df["retailer_count"] = df["market"].str.count(",") + 1

# Reorder columns
preferred_order = [
    "case_number",
    "link_to_h2a",
    "farm",
    "market",
    "best_address",
    "latitude",
    "longitude",
    "location",
    "year",
    "recruiter",
    "person",
    "hit_or_no_hit",
    "certainty",
    "online_presence",
    "has_facebook",
    "retailer_count",
]
df = df[
    [c for c in preferred_order if c in df.columns]
    + [c for c in df.columns if c not in preferred_order]
]

# Save cleaned dataset
df.to_csv("ONION-MASTER.csv", index=False)

# Print
print(df[["case_number", "best_address", "latitude", "longitude"]].head())

          case_number                                      best_address  \
0  H-300-20013-248635             1104 Maynard Ave., Nicholls, GA 31554   
1  H-300-20002-228933                 213 PINECONE ROAD, Alma, GA 31510   
2  H-300-22014-831909  211 Dorothy Burnham Road Nicholls, Georgia 31554   
3  H-300-20357-975271                 931 SA HARDEN RD, Lyons, GA 30436   
4  H-300-21006-998351                 183 MARIGOLD LANE, Alma, GA 31510   

    latitude  longitude  
0  31.517422 -82.634860  
1  31.499044 -82.500469  
2  31.490785 -82.631934  
3  32.135268 -82.313271  
4  31.485531 -82.444244  


/var/folders/c0/9xhm71h569xfdfjm08qm_pp80000gn/T/ipykernel_77541/4210665787.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["certainty"].fillna(0, inplace=True)
/var/folders/c0/9xhm71h569xfdfjm08qm_pp80000gn/T/ipykernel_77541/4210665787.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always beha